In [2]:
import numpy as np
from math import pow
import scipy.optimize

## 1 载入原始数据

### 获取基站个数sta_num（第一行），终端个数dev_num（第二行），数据维度（第三行）

In [3]:
handle_list=[]
with open('./Dataset_2016c/sample_case001_input.txt','r') as handle:
    for line in handle:
        handle_list.append(line.strip('\n'))
sta_num = int(handle_list[0])
dev_num = int(handle_list[1])
dimension = int(handle_list[2])

### 构建基站位置列表

In [4]:
sta_list = []
for i in range(3,3+sta_num):
    sta_list.append([float(x) for x in handle_list[i].split('\t') if x != ''])

### 构建终端TOA列表

In [5]:
toa_list = []
for i in range(3+sta_num,3+sta_num+dev_num):
    toa_list.append([float(x) for x in handle_list[i].split('\t') if x != ''])

## 定位基本模型

### 构建矩阵A、X、Y，并用最小二乘法求解最小值
![矩阵](Dataset_2016c/tutorial_img/matrix.png)

In [9]:
def residual(p,mat_A,mat_Y):
    mat_temp = np.dot(mat_A,[p[0],p[1],p[2]])-mat_Y
    # delta = np.sum(np.multiply(mat_temp,mat_temp))
    delta = np.dot(mat_temp.T,mat_temp)
    return delta

handle_list=[]
with open('./Dataset_2016c/sample_case001_ans.txt','r') as handle:
    for line in handle:
        handle_list.append(line.strip('\n'))
ans_list = []
for line in handle_list:
    ans_list.append([float(x) for x in line.split('\t') if x != ''])

wave_spread_speed = 3e8
for k_2 in range(dev_num):
    mat_A,mat_Y = [],[]
    for k in range(sta_num):
        L_k = wave_spread_speed * toa_list[k_2][k]
        if L_k <= 200:
            x_k,y_k,z_k = sta_list[k][0],sta_list[k][1],sta_list[k][2]
            K_k = x_k*x_k+y_k*y_k+z_k*z_k
            break
    if not (x_k and y_k and z_k):
        print("{}:Bad signal!".format(k_2))
        continue
    for i in range(sta_num):
        L_i = wave_spread_speed * toa_list[k_2][i]
        if L_i <= float(200/.85): # 假设每个基站的通信半径为200米，考虑到NLOS，除以折减系数（超过范围虽然有测量数据，但无效）
            if L_i > 200:
                L_i *= .85
            x_i,y_i,z_i = sta_list[i][0],sta_list[i][1],sta_list[i][2]
            mat_A.append([x_i-x_k,y_i-y_k,z_i-z_k])
            K_i = x_i*x_i+y_i*y_i+z_i*z_i    
            mat_Y.append(float(K_i-L_i*L_i-K_k+L_k*L_k))
    if len(mat_Y) <4:
        print("{}:Not enough valid TOA data! Available: {}".format(k_2,len(mat_Y)))
        continue
    # 以TOA最小值对应的基站坐标为初始值
    p_init = sta_list[toa_list[k_2].index(min(toa_list[k_2]))]
    # print(min(toa_list[k_2]),toa_list[k_2].index(min(toa_list[k_2])),p)
    cons = ({'type': 'ineq', 'fun': lambda x: 40000-pow(x[0]-p_init[0],2)-pow(x[1]-p_init[1],2)-pow(x[2]-p_init[2],2)})
    param=scipy.optimize.minimize(residual,p_init,args=(mat_A,mat_Y),method='SLSQP',constraints=cons)
    print("{}:{} ({}); Answer: {}".format(k_2,np.round(param.x,2),len(mat_Y),ans_list[k_2]))

0:[ -72.19   16.17 -193.3 ] (10); Answer: [-21.19, 4.48, 1.48]
1:[-194.86  135.76   12.61] (7); Answer: [-81.14, 58.24, 1.38]
2:[-133.31 -378.83    7.17] (6); Answer: [-96.13, -215.22, 1.02]
3:[-473.24   -8.23    7.61] (6); Answer: [-296.06, -20.15, 1.59]
4:[ 174.53 -222.17   51.05] (7); Answer: [86.29, -111.06, 1.66]
5:[ 25.9  444.4    7.63] (4); Answer: [-31.26, 244.68, 1.3]
6:[-528.14  -32.32    7.84] (6); Answer: [-286.47, -38.75, 1.73]
7:Not enough valid TOA data! Available: 3
8:Not enough valid TOA data! Available: 3
9:[480.15 156.62   4.14] (6); Answer: [255.34, 66.63, 1.21]
10:[158.05 389.92   6.97] (7); Answer: [104.46, 203.54, 1.95]
11:[-207.41  122.98    3.17] (5); Answer: [-120.39, 92.32, 1.72]
12:[175.35 371.37   6.91] (8); Answer: [114.52, 198.68, 1.33]
13:[-458.51  -37.63    6.85] (5); Answer: [-315.97, -70.5, 1.36]
14:[-450.09   73.02    8.38] (5); Answer: [-257.8, 45.43, 1.32]
15:[ -54.74  -90.81 -176.55] (10); Answer: [-18.52, -30.78, 1.57]
16:[-609.95   44.2     9.69

177:[   2.99 -140.57 -163.32] (9); Answer: [8.49, -62.13, 1.02]
178:[413.71 196.72 -68.64] (6); Answer: [322.12, 119.04, 1.18]
179:[-499.12  147.9    82.83] (4); Answer: [-229.33, 73.33, 1.6]
180:[110.85  -8.97 173.69] (11); Answer: [30.42, -3.17, 1.09]
181:[-364.17 -259.14   14.95] (4); Answer: [-229.1, -194.12, 1.64]
182:Not enough valid TOA data! Available: 2
183:Not enough valid TOA data! Available: 3
184:Not enough valid TOA data! Available: 2
185:[503.58  -2.12   4.24] (5); Answer: [294.89, -5.43, 1.42]
186:[-464.84 -139.      7.43] (5); Answer: [-268.48, -95.98, 1.93]
187:[458.51 166.68   4.15] (6); Answer: [261.5, 65.54, 1.48]
188:[107.52  36.75 164.03] (11); Answer: [28.55, 9.24, 1.96]
189:[503.5   31.94  -4.62] (4); Answer: [321.4, 59.1, 1.95]
190:[-162.91   94.61  138.95] (9); Answer: [-72.25, 47.75, 1.87]
191:[ 104.91  245.62 -127.97] (6); Answer: [66.8, 137.12, 1.44]
192:[287.56 364.82   7.85] (4); Answer: [238.71, 204.31, 1.26]
193:[354.38  62.91   3.64] (7); Answer: [190

359:[-149.1   293.03    6.58] (4); Answer: [-37.08, 158.07, 1.13]
360:Not enough valid TOA data! Available: 1
361:[ 230.51 -130.06   21.1 ] (7); Answer: [103.84, -91.88, 1.12]
362:[-166.67  134.88   89.81] (7); Answer: [-85.87, 78.88, 1.02]
363:[ 16.78 444.42   8.47] (5); Answer: [-10.65, 221.55, 1.66]
364:Not enough valid TOA data! Available: 2
365:Not enough valid TOA data! Available: 2
366:[502.93 -28.     4.25] (5); Answer: [292.07, -22.85, 1.52]
367:[284.87  59.55 150.66] (8); Answer: [144.88, 47.38, 1.78]
368:[ 382.73 -109.33    2.55] (6); Answer: [195.52, -57.28, 1.47]
369:Not enough valid TOA data! Available: 2
370:[-497.02 -224.78    8.26] (5); Answer: [-276.44, -122.59, 1.37]
371:[503.6  36.2   6. ] (6); Answer: [263.18, 18.59, 1.88]
372:Not enough valid TOA data! Available: 2
373:Not enough valid TOA data! Available: 2
374:Not enough valid TOA data! Available: 2
375:[500.02 -48.45   4.16] (5); Answer: [267.79, -39.64, 1.2]
376:Not enough valid TOA data! Available: 2
377:Not 

510:[-384.99  -66.39    5.1 ] (6); Answer: [-207.83, -51.71, 1.47]
511:Not enough valid TOA data! Available: 1
512:[ -34.42 -471.02    4.54] (5); Answer: [19.37, -256.32, 1.34]
513:[-143.7   -63.58 -167.54] (10); Answer: [-51.0, -16.47, 1.05]
514:[351.53 296.    96.84] (6); Answer: [172.19, 164.45, 1.76]
515:[-324.85 -142.   -100.05] (9); Answer: [-161.26, -58.27, 1.41]
516:[-467.69 -119.51    7.42] (5); Answer: [-301.73, -112.2, 1.23]
517:[ 474.91 -171.45    5.13] (5); Answer: [277.26, -93.55, 1.98]
518:Not enough valid TOA data! Available: 2
519:[310.99 452.2    5.63] (4); Answer: [250.32, 243.38, 1.35]
520:[ 476.91 -161.19    4.79] (5); Answer: [244.79, -70.49, 1.92]
521:Not enough valid TOA data! Available: 3
522:[409.52 246.19  10.49] (6); Answer: [293.59, 128.75, 1.93]
523:[ -75.94   53.53 -184.45] (12); Answer: [-47.02, 31.32, 1.63]
524:Not enough valid TOA data! Available: 2
525:Not enough valid TOA data! Available: 1
526:[504.79  38.71   6.07] (5); Answer: [272.0, 19.4, 1.26]


714:[-230.24 -244.01 -118.97] (11); Answer: [-106.84, -115.96, 1.25]
715:[ 437.17 -251.37    5.73] (5); Answer: [259.5, -123.42, 1.76]
716:[-472.98   -4.73    7.19] (5); Answer: [-252.24, -44.19, 1.19]
717:Not enough valid TOA data! Available: 2
718:Not enough valid TOA data! Available: 2
719:[387.   287.07   7.19] (6); Answer: [224.49, 150.04, 1.61]
720:Not enough valid TOA data! Available: 1
721:[-55.99 424.85  44.56] (5); Answer: [24.62, 199.68, 1.32]
722:Not enough valid TOA data! Available: 1
723:[350.97  73.76   3.54] (7); Answer: [213.74, 35.85, 1.35]
724:[2.6371e+02 4.4380e+02 2.6000e-01] (5); Answer: [153.66, 317.47, 1.56]
725:[-50.74 348.25   8.98] (4); Answer: [-64.28, 219.88, 1.79]
726:Not enough valid TOA data! Available: 2
727:Not enough valid TOA data! Available: 2
728:[-241.12    3.93 -188.38] (5); Answer: [-149.19, 25.57, 1.72]
729:[  -9.98 -262.48 -132.84] (9); Answer: [15.47, -117.81, 1.91]
730:[-170.95 -365.21    7.61] (6); Answer: [-106.85, -213.04, 1.47]
731:[ 441

935:[290.33 -70.62   5.72] (6); Answer: [129.75, -56.5, 1.74]
936:[-490.61  -74.46    7.96] (5); Answer: [-255.22, -60.32, 1.63]
937:[-198.4  -346.17  -27.27] (8); Answer: [-89.47, -175.93, 1.24]
938:Not enough valid TOA data! Available: 2
939:Not enough valid TOA data! Available: 1
940:[380.46 251.6    6.57] (8); Answer: [218.59, 128.08, 1.24]
941:[369.36  49.48   9.84] (7); Answer: [216.9, 47.66, 1.15]
942:Not enough valid TOA data! Available: 3
943:Not enough valid TOA data! Available: 2
944:Not enough valid TOA data! Available: 2
945:[-369.43 -112.28    3.02] (9); Answer: [-193.95, -71.72, 1.41]
946:[-466.58  -71.99    7.07] (5); Answer: [-279.15, -75.33, 1.29]
947:[232.29  96.   130.85] (9); Answer: [112.93, 58.04, 1.26]
948:[347.22 -93.43   3.04] (6); Answer: [209.53, -79.44, 1.93]
949:[-402.49 -249.25    7.18] (5); Answer: [-255.72, -139.44, 1.05]
950:[ 151.84 -249.      6.41] (7); Answer: [91.21, -135.61, 1.59]
951:Not enough valid TOA data! Available: 1
952:[-117.   -382.22   

1078:[ 76.73 278.7   46.7 ] (7); Answer: [18.31, 111.17, 1.57]
1079:Not enough valid TOA data! Available: 2
1080:[-150.53 -313.49    6.44] (6); Answer: [-111.45, -222.48, 1.35]
1081:[  70.2  -157.37 -180.46] (9); Answer: [29.21, -61.86, 1.03]
1082:Not enough valid TOA data! Available: 0
1083:[-397.4  -338.14   12.47] (8); Answer: [-185.32, -151.63, 1.71]
1084:Not enough valid TOA data! Available: 1
1085:[-203.57  123.78    6.07] (4); Answer: [-134.61, 110.41, 1.4]
1086:[-203.84 -302.95 -104.14] (11); Answer: [-96.53, -139.73, 1.1]
1087:[323.39 -62.51 114.86] (9); Answer: [174.0, -26.48, 1.0]
1088:[-52.23 306.67 124.03] (5); Answer: [17.41, 161.38, 1.77]
1089:[486.93 101.76   6.11] (5); Answer: [286.37, 41.74, 1.89]
1090:Not enough valid TOA data! Available: 2
1091:[221.7   32.03 144.83] (11); Answer: [69.38, 17.44, 1.29]
1092:[ -35.39 -282.64 -100.76] (9); Answer: [9.73, -123.93, 1.3]
1093:[ 326.21 -253.22  122.07] (6); Answer: [170.4, -129.26, 1.96]
1094:Not enough valid TOA data! Ava